# Action Detection
- Using MediaPipe to detect only using RH and LH keypoints

Notes for future improvements:
1. FACEMESH can map over 1000 keypoints on the face, and pose can map 132. 
    - FACEMESH can allow inclusion of facial expressions that can be paired with handsigns to help better translate the word, as there are some handsigns that can have another meaning based on context/ emotions
    - POSE can also be used in real life scenarios to pair with handsigns (filming the entire person using a phone) to improve accuracy
    * However, as our dataset is very small, we will not be using these points as they negatively influence the prediction. Implementation will need more work in this area
    
2. Threading to implement parallelism on prediction and text to speech from webcam output

## Setup Dependencies, Libraries, Helper Functions

In [ ]:
# Install dependencies
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib pyttsx3

In [10]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pyttsx3

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# Define detection and drawing functions
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(60,20,220), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(0,255,30), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(60,20,220), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(0,255,30), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [59]:
# Check camera set up
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Set mediapipe model: instantiate the model to fit into detection function
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read frame from webcam, return value, frame: image from webcam
        ret, frame = cap.read()

        # Feed webcam image to mediapipe to Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [30]:
cap.release()
cv2.destroyAllWindows()

## Setup Folders for Data Collection


In [5]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('hire') 

# Actions that we try to detect
actions = np.array(['please', 'hire', 'me', 'google'])

# Define number of videos to record
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

In [6]:
actions.shape[0]

4

In [116]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## Collect keypoints for training

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length, each frame, 30 frames of keypoints for each video
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence+50), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
cap.release()
cv2.destroyAllWindows()

## Training Model

### Preprocess Data

In [97]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [132]:
label_map = {label:num for num, label in enumerate(actions)}

In [133]:
label_map

{'please': 0, 'hire': 1, 'me': 2, 'google': 3}

In [134]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [135]:
# Check shape: (videos, frames, keypoints), adjust LSTM input accordingly
np.array(sequences).shape

(192, 30, 126)

In [136]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [137]:
X_train=X
y_train=y

In [ ]:
# Train test split (skip for more train data)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_seed=0)

### Model Training

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import Model, callbacks
# from tensorflow.keras.callbacks import TensorBoard

In [16]:
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)

In [138]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [139]:
early_stopping = EarlyStopping(monitor='categorical_accuracy', min_delta=0.001, patience=100, verbose=1, mode='auto', restore_best_weights=True)

In [140]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [141]:
model.fit(X_train, y_train, epochs=1000, callbacks=[early_stopping])

Epoch 1/1000
6/6 [==============================] - 0s 62ms/step - loss: 1.3159 - categorical_accuracy: 0.3333
Epoch 2/1000
6/6 [==============================] - 0s 58ms/step - loss: 1.3038 - categorical_accuracy: 0.3021 0s - loss: 1.3682 - categorical_accuracy: 
Epoch 3/1000
6/6 [==============================] - 0s 66ms/step - loss: 1.2561 - categorical_accuracy: 0.3490
Epoch 4/1000
6/6 [==============================] - 0s 61ms/step - loss: 1.2633 - categorical_accuracy: 0.4948
Epoch 5/1000
6/6 [==============================] - 0s 60ms/step - loss: 1.1798 - categorical_accuracy: 0.3490
Epoch 6/1000
6/6 [==============================] - 0s 63ms/step - loss: 1.0378 - categorical_accuracy: 0.5052
Epoch 7/1000
6/6 [==============================] - 0s 55ms/step - loss: 0.9170 - categorical_accuracy: 0.5729
Epoch 8/1000
6/6 [==============================] - 0s 60ms/step - loss: 0.8577 - categorical_accuracy: 0.5052
Epoch 9/1000
6/6 [==============================] - 0s 65ms/step - lo

6/6 [==============================] - 0s 65ms/step - loss: 0.8669 - categorical_accuracy: 0.7812
Epoch 146/1000
6/6 [==============================] - 0s 66ms/step - loss: 1.0929 - categorical_accuracy: 0.7396
Epoch 147/1000
6/6 [==============================] - 0s 65ms/step - loss: 1.2133 - categorical_accuracy: 0.4167
Epoch 148/1000
6/6 [==============================] - 0s 67ms/step - loss: 0.9281 - categorical_accuracy: 0.6094
Epoch 149/1000
6/6 [==============================] - 0s 67ms/step - loss: 0.8871 - categorical_accuracy: 0.6562
Epoch 150/1000
6/6 [==============================] - 0s 67ms/step - loss: 0.8306 - categorical_accuracy: 0.7448
Epoch 151/1000
6/6 [==============================] - 0s 56ms/step - loss: 0.7565 - categorical_accuracy: 0.7812
Epoch 152/1000
6/6 [==============================] - 0s 68ms/step - loss: 0.6768 - categorical_accuracy: 0.8281
Epoch 153/1000
6/6 [==============================] - 0s 64ms/step - loss: 0.6115 - categorical_accuracy: 0.828

In [142]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 30, 64)            48896     
_________________________________________________________________
lstm_37 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_38 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_36 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_37 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_38 (Dense)             (None, 4)                 132       
Total params: 203,492
Trainable params: 203,492
Non-trainable params: 0
_______________________________________________

In [143]:
model.save('action_hireme.h5')

In [65]:
# model.load_weights('hire_v2.h5')

## Test Detection, Demo

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define actions
actions = np.array(['hire', 'me', 'please', 'Google'])

# Model import
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.load_weights('action_hireme.h5')


In [18]:
# Testing implementation of text to speech
s = pyttsx3.init()           #for tts
s.setProperty('rate', 100)     # setting up new voice rate
voices = s.getProperty('voices') 
s.setProperty('voice', voices[0].id)  #change voice
data = "hire"  
s.say(data)  
s.runAndWait()  

In [12]:
# Test model with real time webcam video

sequence = []        #collect the 30 frames to generate a prediction
sentence = []        #concat history of detections
predictions = []
max_words = 8
threshold = 0.8

cap = cv2.VideoCapture(0)
tts = pyttsx3.init()           #for tts
tts.setProperty('rate', 100)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        
        if results.right_hand_landmarks or results.left_hand_landmarks:
            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
        
            # Make prediction once there is 30 frames in the sequence
            if len(sequence) == 30: 
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predictions.append(np.argmax(res))
                
                # Vizualisation logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:   
                                sentence.append(actions[np.argmax(res)])
                                tts.say(actions[np.argmax(res)])
                                tts.runAndWait()
                        else:
                            sentence.append(actions[np.argmax(res)])
                            tts.say(actions[np.argmax(res)])
                            tts.runAndWait()
                            
        # Skip when no hands are detected
        else:   
            pass
            
            if len(sentence) > max_words:          #just grab the last 5 values when sentence has more than 6 words
                sentence = sentence[-max_words:]
            
        cv2.rectangle(image, (0,0), (1280, 40), (245, 139, 76), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),                                 # (left-right, up-down)
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [78]:
cap.release()
cv2.destroyAllWindows()

## Mediapipe Holistic Demo
- Just to demonstrate mediapipe holistic with its face, pose and hand detection

In [13]:
# Video demo for Mediapipe Holistic
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(60,20,220), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(255,144,30), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(60,20,220), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(0,255,30), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(60,20,220), thickness=2, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(0,255,30), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Check camera set up
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Set mediapipe model: instantiate the model to fit into detection function
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read frame from webcam, return value, frame: image from webcam
        ret, frame = cap.read()

        # Feed webcam image to mediapipe to Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

## Convert model to tflite

In [80]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('sentence1.tflite', 'wb') as f:
    f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: C:\Users\wushen\AppData\Local\Temp\tmp_mad_k0k\assets


INFO:tensorflow:Assets written to: C:\Users\wushen\AppData\Local\Temp\tmp_mad_k0k\assets
